# Distributed SQL with Dask

In [ ]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)

In [ ]:
from blazingsql import BlazingContext

bc = BlazingContext(dask_client=client, network_interface='lo')

In [ ]:
bc.s3('bsql', bucket_name='blazingsql-colab')

In [ ]:
files_10 = [f's3://bsql/taxi_parquet/yellow_tripdata_2010-0{i}.parquet' if i < 10 
              else f's3://bsql/taxi_parquet/yellow_tripdata_2010-{i}.parquet' for i in range(1, 13)]

files_11 = [f's3://bsql/taxi_parquet/yellow_tripdata_2011-0{i}.parquet' if i < 10 
              else f's3://bsql/taxi_parquet/yellow_tripdata_2011-{i}.parquet' for i in range(1, 13)]

f = files_10 + files_11

In [ ]:
bc.create_table('taxi', f)

In [ ]:
pi = 3.141592653589793
o_shift = 2 * pi * 6378137 / 2

query = f'''
        SELECT 
            dropoff_longitude * {o_shift} / 180 AS dropoff_x, 
            LOG10(TAN(((90 + dropoff_latitude) * {pi} / 360))) / {pi} / 180 * {o_shift} / 180 AS dropoff_y
        FROM 
            taxi
            WHERE
                pickup_longitude < -73.75 AND pickup_longitude > -74.15
                AND dropoff_longitude < -73.75 AND dropoff_longitude > -74.15
                AND pickup_latitude > 40.68 AND pickup_latitude < 40.84
                AND dropoff_latitude > 40.68 AND dropoff_latitude < 40.84
                '''

In [ ]:
from colorcet import fire
from datashader import Canvas, transfer_functions as tf

In [ ]:
agg = Canvas().points(bc.sql(query), 'dropoff_x', 'dropoff_y')

tf.set_background(tf.shade(agg, cmap=fire), "black")